In [47]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [1]:
#!pip install catboost

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
#myfolder = "./"
myfolder = "../data/CMaps/"

# **Columns' names**

In [4]:
#Columns' names
'''
1)  unit number
2)	time, in cycles
3)	operational setting 1
4)	operational setting 2
5)	operational setting 3
6)	sensor measurement  1
7)	sensor measurement  2
...
26)	sensor measurement  21
'''
unitNames = ['UnitNumber']
timeCycles = ["TimeInCycles"]
operSets = ["OperSet"+str(i) for i in range(1,4)] # 1,2 et 3
sensorMes = ["SensorMes"+str(j) for j in range(1, 22)] # de 1 à 21
columnsNames = unitNames + timeCycles + operSets +sensorMes

# **Datasets loading**

In [5]:
def data_loading(x):
  train_path = myfolder + "train_"+ x +".txt"
  test_path = myfolder + "test_"+ x +".txt"
  rul_path = myfolder + "RUL_"+ x +".txt"
  train = pd.read_csv(train_path, delim_whitespace=True, names=columnsNames)
  test = pd.read_csv(test_path, delim_whitespace=True, names=columnsNames)
  rul = pd.read_csv(rul_path, delim_whitespace=True, names=["RUL_FD"])
  return train, test, rul

train_fd001, test_fd001, rul_fd001 = data_loading("FD001")
train_fd002, test_fd002, rul_fd002 = data_loading("FD002")
train_fd003, test_fd003, rul_fd003 = data_loading("FD003")
train_fd004, test_fd004, rul_fd004 = data_loading("FD004")

In [6]:
# Forcer l'affichage de toutes les colonnes
pd.set_option('display.max_columns', None)

train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.0,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.0,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.0,10.48,6.4213


# **RUL column generation for train and test set**

In [83]:
def rul_train_generation(x):
  rul = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  rul.columns = ['UnitNumber', 'max']
  x = x.merge(rul, on=['UnitNumber'], how='left')
  x['RUL'] = x['max'] - x['TimeInCycles']
  x.drop('max', axis=1, inplace=True)
  return x

train_fd001 = rul_train_generation(train_fd001)
train_fd002 = rul_train_generation(train_fd002)
train_fd003 = rul_train_generation(train_fd003)
train_fd004 = rul_train_generation(train_fd004)

In [84]:
train_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.0,10.62,6.3670,320
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.0,24.37,14.6552,319
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.0,10.48,6.4213,318


In [85]:
def rul_test_generation(x, rul):

  rul["UnitNumber"] = rul.index + 1 # +1 pour que UnitNumber demarre de 1 au lieu de 0, car il s'agit du numero des moteur

  x = x.merge(rul, on=['UnitNumber'], how='left')

  max_cycle = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  max_cycle.columns = ['UnitNumber', 'max']
  x = x.merge(max_cycle, on=['UnitNumber'], how='left')
  x['RUL'] = x['RUL_FD'] + x['max'] - x['TimeInCycles']
  x.drop(['max', 'RUL_FD'], axis=1, inplace=True)

  return x

test_fd001 = rul_test_generation(test_fd001, rul_fd001)
test_fd002 = rul_test_generation(test_fd002, rul_fd002)
test_fd003 = rul_test_generation(test_fd003, rul_fd003)
test_fd004 = rul_test_generation(test_fd004, rul_fd004)

In [86]:
test_fd004.head(3)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
0,1,1,20.0072,0.700,100.0,491.19,606.67,1481.04,1227.81,9.35,13.60,332.52,2323.67,8704.98,1.07,43.83,313.03,2387.78,8048.98,9.2229,0.02,362,2324,100.00,24.31,14.7007,251
1,1,2,24.9984,0.620,60.0,462.54,536.22,1256.17,1031.48,7.05,9.00,174.46,1915.21,7999.94,0.93,36.11,163.61,2028.09,7863.46,10.8632,0.02,306,1915,84.93,14.36,8.5748,250
2,1,3,42.0000,0.842,100.0,445.00,549.23,1340.13,1105.88,3.91,5.69,137.34,2211.93,8305.38,1.01,41.52,129.98,2387.95,8071.13,9.3960,0.02,328,2212,100.00,10.39,6.4365,249


# **Bivariate Correlation Analysis**

In [87]:
'''def correl(df):
    plt.figure(figsize=(10, 8))
    cmap = sns.diverging_palette(500, 10, as_cmap=True)
    sns.heatmap(train_fd001.corr(), cmap = cmap, center=0, annot=False, square=True)'''
def correl(df):
    return df.corr()


def correlation_analyz(df, seuil):
  # Calculer la corrélation entre les variables avec différentes méthodes
  corr_pearson = df.corr(method='pearson')
  corr_spearman = df.corr(method='spearman')
  corr_kendall = df.corr(method='kendall')

  # Colonnes avec corrélation plus faible que le seuil pour chaque méthode
  colonnes_faible_corr_pearson = corr_pearson.loc[corr_pearson['RUL'].abs() < seuil].index.tolist()
  colonnes_faible_corr_spearman = corr_spearman.loc[corr_spearman['RUL'].abs() < seuil].index.tolist()
  colonnes_faible_corr_kendall = corr_kendall.loc[corr_kendall['RUL'].abs() < seuil].index.tolist()

  # Afficher les colonnes avec corrélation plus faible que le seuil pour chaque méthode
  print("Colonnes avec corrélation plus faible que", seuil, "pour la corrélation de Pearson:")
  print(colonnes_faible_corr_pearson)
  print("\nColonnes avec corrélation plus faible que", seuil, "pour la corrélation de Spearman:")
  print(colonnes_faible_corr_spearman)
  print("\nColonnes avec corrélation plus faible que", seuil, "pour la corrélation de Kendall:")
  print(colonnes_faible_corr_kendall)

In [88]:
correl(train_fd001)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
UnitNumber,1.000000,0.078753,-0.017895,-0.006197,NaN,NaN,0.014133,0.012621,0.026116,NaN,0.026431,-0.032073,0.039929,-0.052053,NaN,0.024707,-0.031882,0.044230,-0.059473,0.022486,NaN,0.013513,NaN,NaN,-0.020589,-0.016465,0.078753
TimeInCycles,0.078753,1.000000,-0.004527,0.016132,NaN,NaN,0.549898,0.543947,0.624577,NaN,0.105980,-0.595914,0.475977,0.443999,NaN,0.634385,-0.611354,0.477523,0.370324,0.588676,NaN,0.566995,NaN,NaN,-0.583597,-0.585923,-0.736241
OperSet1,-0.017895,-0.004527,1.000000,0.011660,NaN,NaN,0.009030,-0.005651,0.009544,NaN,-0.001340,-0.009437,-0.000427,-0.004279,NaN,0.011658,-0.001491,0.002318,-0.004469,0.007652,NaN,0.002599,NaN,NaN,-0.005713,-0.014559,-0.003198
OperSet2,-0.006197,0.016132,0.011660,1.000000,NaN,NaN,0.007266,0.009068,0.014673,NaN,0.014355,-0.016678,0.013285,-0.005425,NaN,0.011690,-0.010923,0.018156,-0.006310,0.014156,NaN,0.012280,NaN,NaN,-0.010554,-0.007846,-0.001948
OperSet3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SensorMes1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SensorMes2,0.014133,0.549898,0.009030,0.007266,NaN,NaN,1.000000,0.602610,0.714949,NaN,0.132242,-0.702136,0.662325,0.273764,NaN,0.740020,-0.724578,0.661792,0.179752,0.675975,NaN,0.629886,NaN,NaN,-0.661841,-0.668050,-0.606484
SensorMes3,0.012621,0.543947,-0.005651,0.009068,NaN,NaN,0.602610,1.000000,0.678413,NaN,0.116039,-0.664595,0.602481,0.322964,NaN,0.695900,-0.680307,0.600963,0.237137,0.639921,NaN,0.600017,NaN,NaN,-0.625941,-0.633901,-0.584520
SensorMes4,0.026116,0.624577,0.009544,0.014673,NaN,NaN,0.714949,0.678413,1.000000,NaN,0.150480,-0.793130,0.746852,0.297429,NaN,0.830136,-0.815591,0.745158,0.190748,0.758459,NaN,0.703499,NaN,NaN,-0.748067,-0.745193,-0.678948
SensorMes5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
correlation_analyz(train_fd001, 0.6)

Colonnes avec corrélation plus faible que 0.6 pour la corrélation de Pearson:
['UnitNumber']

Colonnes avec corrélation plus faible que 0.6 pour la corrélation de Spearman:
['UnitNumber']

Colonnes avec corrélation plus faible que 0.6 pour la corrélation de Kendall:
['UnitNumber', 'TimeInCycles', 'SensorMes2', 'SensorMes4', 'SensorMes7', 'SensorMes11', 'SensorMes12', 'SensorMes15', 'SensorMes17', 'SensorMes20', 'SensorMes21']


In [61]:
# Les colonnes à supprimer
operSets_to_drop = ["OperSet"+str(i) for i in [1, 2]] # 1 et 2
sensorMes_to_drop = ["SensorMes"+str(j) for j in [3, 6, 8, 9, 13, 14]]
cols_to_drop = operSets_to_drop + sensorMes_to_drop

# Suppression dans train et test
drop_cols=True
if(drop_cols == True):
  train_fd001 = train_fd001.drop(cols_to_drop, axis=1)
  test_fd001 = test_fd001.drop(cols_to_drop, axis=1)

In [62]:
train_fd001.columns, len(train_fd001.columns)

(Index(['UnitNumber', 'TimeInCycles', 'OperSet3', 'SensorMes1', 'SensorMes2',
        'SensorMes4', 'SensorMes5', 'SensorMes7', 'SensorMes10', 'SensorMes11',
        'SensorMes12', 'SensorMes15', 'SensorMes16', 'SensorMes17',
        'SensorMes18', 'SensorMes19', 'SensorMes20', 'SensorMes21', 'RUL'],
       dtype='object'),
 19)

In [89]:
correl(train_fd002)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
UnitNumber,1.000000,0.014608,-0.003084,-0.005160,0.004777,0.001659,0.003531,0.006220,0.007947,0.002592,0.003033,0.003528,0.004669,0.003368,0.006838,0.009307,0.003562,0.004748,0.001573,-0.004108,0.010823,0.006432,0.004695,0.004777,0.003119,0.003103,0.014608
TimeInCycles,0.014608,1.000000,0.000039,0.000750,-0.005057,-0.001011,0.003753,0.024635,0.037108,-0.000516,-0.000691,-0.003254,-0.004035,0.022238,-0.004292,0.042219,-0.003088,-0.004331,0.071592,0.035143,0.067857,0.025085,-0.004642,-0.005057,-0.006865,-0.006766,-0.733383
OperSet1,-0.003084,0.000039,1.000000,0.943005,-0.028488,-0.964414,-0.867452,-0.787862,-0.840737,-0.987344,-0.976459,-0.951622,-0.444419,-0.776655,-0.827911,-0.605517,-0.951201,-0.028653,-0.263387,0.460837,-0.838749,-0.790552,-0.443924,-0.028488,-0.962036,-0.962058,0.001135
OperSet2,-0.005160,0.000750,0.943005,1.000000,-0.065350,-0.884061,-0.808822,-0.759658,-0.836673,-0.932606,-0.928264,-0.917853,-0.439033,-0.752442,-0.885541,-0.622257,-0.917718,-0.065437,-0.355461,0.520009,-0.888615,-0.762704,-0.438600,-0.065350,-0.920772,-0.920777,0.000047
OperSet3,0.004777,-0.005057,-0.028488,-0.065350,1.000000,0.164334,0.480073,0.620839,0.544563,0.113635,0.198092,0.306989,0.902746,0.634739,0.508645,0.800034,0.308334,0.999998,0.927377,-0.884175,0.295212,0.617185,0.902999,1.000000,0.276544,0.276479,0.005761
SensorMes1,0.001659,-0.001011,-0.964414,-0.884061,0.164334,1.000000,0.944089,0.870963,0.898002,0.986372,0.986424,0.973142,0.572652,0.861836,0.826591,0.706111,0.972867,0.164474,0.352782,-0.542743,0.793789,0.873265,0.572171,0.164334,0.977703,0.977718,-0.000023
SensorMes2,0.003531,0.003753,-0.867452,-0.808822,0.480073,0.944089,1.000000,0.982225,0.981047,0.915808,0.944072,0.968620,0.810662,0.978554,0.905135,0.895735,0.968828,0.480202,0.623963,-0.777953,0.805280,0.983065,0.810312,0.480073,0.962425,0.962416,-0.004928
SensorMes3,0.006220,0.024635,-0.787862,-0.759658,0.620839,0.870963,0.982225,1.000000,0.989565,0.842951,0.884795,0.929054,0.895718,0.997806,0.928979,0.960683,0.929534,0.620970,0.751865,-0.875928,0.804821,0.998680,0.895446,0.620839,0.917144,0.917125,-0.026942
SensorMes4,0.007947,0.037108,-0.840737,-0.836673,0.544563,0.898002,0.981047,0.989565,1.000000,0.884242,0.919684,0.956731,0.843956,0.987319,0.961586,0.936763,0.957146,0.544693,0.715493,-0.846726,0.859130,0.990213,0.843647,0.544563,0.946478,0.946469,-0.040978
SensorMes5,0.002592,-0.000516,-0.987344,-0.932606,0.113635,0.986372,0.915808,0.842951,0.884242,1.000000,0.996311,0.979787,0.524331,0.832968,0.843905,0.673774,0.979487,0.113778,0.330514,-0.525384,0.824095,0.845626,0.523827,0.113635,0.985714,0.985727,-0.000758


In [64]:
correlation_analyz(train_fd002, 0.03)

Colonnes avec corrélation plus faible que 0.03 pour la corrélation de Pearson:
['UnitNumber', 'OperSet1', 'OperSet2', 'OperSet3', 'SensorMes1', 'SensorMes2', 'SensorMes3', 'SensorMes5', 'SensorMes6', 'SensorMes7', 'SensorMes8', 'SensorMes9', 'SensorMes10', 'SensorMes12', 'SensorMes13', 'SensorMes17', 'SensorMes18', 'SensorMes19', 'SensorMes20', 'SensorMes21']

Colonnes avec corrélation plus faible que 0.03 pour la corrélation de Spearman:
['UnitNumber', 'OperSet1', 'OperSet2', 'OperSet3', 'SensorMes1', 'SensorMes5', 'SensorMes10', 'SensorMes18', 'SensorMes19']

Colonnes avec corrélation plus faible que 0.03 pour la corrélation de Kendall:
['UnitNumber', 'OperSet1', 'OperSet2', 'OperSet3', 'SensorMes1', 'SensorMes5', 'SensorMes10', 'SensorMes18', 'SensorMes19']


In [65]:
# Les colonnes à supprimer
operSets_to_drop = ["OperSet"+str(i) for i in range(1,4)] # 1,2 et 3
sensorMes_to_drop = ["SensorMes"+str(j) for j in [1, 5, 10, 18, 19]]
cols_to_drop = operSets_to_drop + sensorMes_to_drop

# Suppression dans train et test
drop_cols=True
if(drop_cols == True):
  train_fd002 = train_fd002.drop(cols_to_drop, axis=1)
  test_fd002 = test_fd002.drop(cols_to_drop, axis=1)

In [66]:
train_fd002.columns, len(train_fd002.columns)

(Index(['UnitNumber', 'TimeInCycles', 'SensorMes2', 'SensorMes3', 'SensorMes4',
        'SensorMes6', 'SensorMes7', 'SensorMes8', 'SensorMes9', 'SensorMes11',
        'SensorMes12', 'SensorMes13', 'SensorMes14', 'SensorMes15',
        'SensorMes16', 'SensorMes17', 'SensorMes20', 'SensorMes21', 'RUL'],
       dtype='object'),
 19)

In [90]:
correl(train_fd003)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
UnitNumber,1.000000,-0.077196,-0.002637,-0.016477,NaN,NaN,0.021424,0.027339,0.024376,NaN,0.070735,0.064626,0.050085,0.052341,0.029120,0.029192,0.065447,0.048920,0.051634,-0.035267,NaN,0.027530,NaN,NaN,0.047705,0.046634,-0.077196
TimeInCycles,-0.077196,1.000000,-0.003205,0.003410,NaN,NaN,0.300798,0.373506,0.361937,NaN,-0.418590,0.531015,0.438111,0.507838,0.528220,0.407412,0.538197,0.437563,0.488564,-0.400594,NaN,0.386185,NaN,NaN,0.304585,0.311020,-0.482744
OperSet1,-0.002637,-0.003205,1.000000,0.001226,NaN,NaN,-0.002098,-0.004154,0.000811,NaN,-0.000051,-0.002498,-0.001344,-0.007728,-0.005541,-0.003287,-0.003288,-0.002467,-0.008939,0.007104,NaN,0.001679,NaN,NaN,-0.005606,-0.001452,0.003700
OperSet2,-0.016477,0.003410,0.001226,1.000000,NaN,NaN,0.005117,0.005186,0.004128,NaN,-0.006029,-0.002446,0.000628,0.012311,-0.002698,0.002195,-0.001333,-0.001413,0.015843,0.002108,NaN,0.003078,NaN,NaN,-0.001032,-0.003564,-0.001204
OperSet3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SensorMes1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SensorMes2,0.021424,0.300798,-0.002098,0.005117,NaN,NaN,1.000000,0.640503,0.745167,NaN,0.314799,0.124167,0.533915,0.441283,0.256388,0.762269,0.141785,0.532745,0.343954,0.232947,NaN,0.670062,NaN,NaN,-0.246286,-0.241318,-0.590564
SensorMes3,0.027339,0.373506,-0.004154,0.005186,NaN,NaN,0.640503,1.000000,0.716890,NaN,0.269463,0.282007,0.637926,0.535074,0.367705,0.746093,0.298941,0.636513,0.454205,0.076820,NaN,0.677216,NaN,NaN,-0.091851,-0.089035,-0.624236
SensorMes4,0.024376,0.361937,0.000811,0.004128,NaN,NaN,0.745167,0.716890,1.000000,NaN,0.319139,0.181976,0.601272,0.509782,0.330834,0.854030,0.202106,0.601254,0.404686,0.216773,NaN,0.749907,NaN,NaN,-0.235016,-0.230134,-0.657224
SensorMes5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
correlation_analyz(train_fd003, 0.4)

Colonnes avec corrélation plus faible que 0.4 pour la corrélation de Pearson:
['UnitNumber', 'OperSet1', 'OperSet2', 'SensorMes6', 'SensorMes7', 'SensorMes10', 'SensorMes12', 'SensorMes15', 'SensorMes20', 'SensorMes21']

Colonnes avec corrélation plus faible que 0.4 pour la corrélation de Spearman:
['UnitNumber', 'OperSet1', 'OperSet2', 'SensorMes6', 'SensorMes7', 'SensorMes12', 'SensorMes15', 'SensorMes20', 'SensorMes21']

Colonnes avec corrélation plus faible que 0.4 pour la corrélation de Kendall:
['UnitNumber', 'OperSet1', 'OperSet2', 'SensorMes6', 'SensorMes7', 'SensorMes10', 'SensorMes12', 'SensorMes14', 'SensorMes15', 'SensorMes20', 'SensorMes21']


In [69]:
# Les colonnes à supprimer
operSets_to_drop = ["OperSet"+str(i) for i in [1, 2]] # 1 et 2
sensorMes_to_drop = ["SensorMes"+str(j) for j in [6, 7, 10, 12, 15, 20, 21]]
cols_to_drop = operSets_to_drop + sensorMes_to_drop

# Suppression dans train et test
drop_cols=True
if(drop_cols == True):
  train_fd003 = train_fd003.drop(cols_to_drop, axis=1)
  test_fd003 = test_fd003.drop(cols_to_drop, axis=1)

In [70]:
train_fd003.columns, len(train_fd003.columns)

(Index(['UnitNumber', 'TimeInCycles', 'OperSet3', 'SensorMes1', 'SensorMes2',
        'SensorMes3', 'SensorMes4', 'SensorMes5', 'SensorMes8', 'SensorMes9',
        'SensorMes11', 'SensorMes13', 'SensorMes14', 'SensorMes16',
        'SensorMes17', 'SensorMes18', 'SensorMes19', 'RUL'],
       dtype='object'),
 18)

In [91]:
correl(train_fd004)

,UnitNumber,TimeInCycles,OperSet1,OperSet2,OperSet3,SensorMes1,SensorMes2,SensorMes3,SensorMes4,SensorMes5,SensorMes6,SensorMes7,SensorMes8,SensorMes9,SensorMes10,SensorMes11,SensorMes12,SensorMes13,SensorMes14,SensorMes15,SensorMes16,SensorMes17,SensorMes18,SensorMes19,SensorMes20,SensorMes21,RUL
UnitNumber,1.000000,-0.003656,0.000171,-0.001140,0.003334,0.001516,0.002333,0.002521,0.001785,0.001064,0.001419,0.001939,0.003423,0.003650,0.002127,0.002493,0.001931,0.003360,0.006958,-0.004081,-0.000835,0.002462,0.003400,0.003334,0.001782,0.001788,-0.003656
TimeInCycles,-0.003656,1.000000,0.000301,0.001033,-0.001814,-0.000688,0.002504,0.023167,0.029306,-0.000489,-0.001291,0.004971,-0.001056,0.028468,0.015330,0.036907,0.005094,-0.000922,0.097872,-0.027760,0.044364,0.022909,-0.001803,-0.001814,0.002463,0.002475,-0.610620
OperSet1,0.000171,0.000301,1.000000,0.943796,-0.028425,-0.964754,-0.868390,-0.787848,-0.839988,-0.987382,-0.976518,-0.951582,-0.444728,-0.776800,-0.823819,-0.605412,-0.951158,-0.028443,-0.263960,0.460800,-0.840823,-0.790694,-0.444336,-0.028425,-0.962043,-0.962052,-0.002380
OperSet2,-0.001140,0.001033,0.943796,1.000000,-0.066257,-0.886123,-0.811613,-0.761372,-0.837055,-0.933693,-0.929548,-0.919006,-0.440914,-0.754337,-0.882565,-0.623179,-0.918868,-0.066236,-0.356085,0.520309,-0.902160,-0.764491,-0.440552,-0.066257,-0.922004,-0.922000,-0.002280
OperSet3,0.003334,-0.001814,-0.028425,-0.066257,1.000000,0.163835,0.478659,0.620181,0.544482,0.113471,0.197793,0.306687,0.902625,0.634634,0.514065,0.798813,0.308038,0.999998,0.928067,-0.881196,0.290752,0.616473,0.902830,1.000000,0.276239,0.276147,0.002303
SensorMes1,0.001516,-0.000688,-0.964754,-0.886123,0.163835,1.000000,0.944439,0.870606,0.897421,0.986561,0.986539,0.973191,0.572469,0.861569,0.823653,0.705775,0.972915,0.163834,0.353450,-0.542375,0.789447,0.872955,0.572078,0.163835,0.977777,0.977791,0.001889
SensorMes2,0.002333,0.002504,-0.868390,-0.811613,0.478659,0.944439,1.000000,0.981750,0.980722,0.916509,0.944587,0.968979,0.809878,0.978305,0.904535,0.894876,0.969187,0.478666,0.624368,-0.776156,0.800320,0.982621,0.809591,0.478659,0.962824,0.962806,-0.004443
SensorMes3,0.002521,0.023167,-0.787848,-0.761372,0.620181,0.870606,0.981750,1.000000,0.989744,0.842817,0.884586,0.929013,0.895268,0.998190,0.929839,0.960787,0.929499,0.620227,0.755184,-0.875041,0.801512,0.998693,0.895021,0.620181,0.917055,0.917020,-0.032924
SensorMes4,0.001785,0.029306,-0.839988,-0.837055,0.544482,0.897421,0.980722,0.989744,1.000000,0.883579,0.919064,0.956314,0.843898,0.987901,0.961264,0.937163,0.956736,0.544531,0.719325,-0.846000,0.858554,0.990407,0.843615,0.544482,0.945999,0.945965,-0.045881
SensorMes5,0.001064,-0.000489,-0.987382,-0.933693,0.113471,0.986561,0.916509,0.842817,0.883579,1.000000,0.996316,0.979719,0.524506,0.832973,0.840662,0.673610,0.979416,0.113473,0.331134,-0.525064,0.824766,0.845583,0.524096,0.113471,0.985677,0.985696,0.001679


In [72]:
correlation_analyz(train_fd004, 0.03)

Colonnes avec corrélation plus faible que 0.03 pour la corrélation de Pearson:
['UnitNumber', 'OperSet1', 'OperSet2', 'OperSet3', 'SensorMes1', 'SensorMes2', 'SensorMes5', 'SensorMes6', 'SensorMes7', 'SensorMes8', 'SensorMes9', 'SensorMes10', 'SensorMes12', 'SensorMes13', 'SensorMes15', 'SensorMes18', 'SensorMes19', 'SensorMes20', 'SensorMes21']

Colonnes avec corrélation plus faible que 0.03 pour la corrélation de Spearman:
['UnitNumber', 'OperSet1', 'OperSet2', 'OperSet3', 'SensorMes1', 'SensorMes5', 'SensorMes7', 'SensorMes12', 'SensorMes15', 'SensorMes18', 'SensorMes19', 'SensorMes20', 'SensorMes21']

Colonnes avec corrélation plus faible que 0.03 pour la corrélation de Kendall:
['UnitNumber', 'OperSet1', 'OperSet2', 'OperSet3', 'SensorMes1', 'SensorMes5', 'SensorMes6', 'SensorMes7', 'SensorMes12', 'SensorMes15', 'SensorMes18', 'SensorMes19', 'SensorMes20', 'SensorMes21']


In [73]:
# Les colonnes à supprimer
operSets_to_drop = ["OperSet"+str(i) for i in range(1,4)] # 1,2 et 3
sensorMes_to_drop = ["SensorMes"+str(j) for j in [1, 5, 6, 7, 12, 15, 18, 19, 20, 21]]
cols_to_drop = operSets_to_drop + sensorMes_to_drop

# Suppression dans train et test
drop_cols=True
if(drop_cols == True):
  train_fd004 = train_fd004.drop(cols_to_drop, axis=1)
  test_fd004 = test_fd004.drop(cols_to_drop, axis=1)

In [74]:
train_fd004.columns, len(train_fd004.columns)

(Index(['UnitNumber', 'TimeInCycles', 'SensorMes2', 'SensorMes3', 'SensorMes4',
        'SensorMes8', 'SensorMes9', 'SensorMes10', 'SensorMes11', 'SensorMes13',
        'SensorMes14', 'SensorMes16', 'SensorMes17', 'RUL'],
       dtype='object'),
 14)

# **RANDOM SAMPLE SELECTION**

In [37]:
'''element_counts = test_fd002[test_fd002['UnitNumber']==1]
len(element_counts)'''

"element_counts = test_fd002[test_fd002['UnitNumber']==1]\nlen(element_counts)"

In [38]:
import random

def selection_aleatoire(df, sample_size, rand_state):
    unique_values = df["UnitNumber"].unique()
    selected_rows = []
    for value in unique_values:
        rows = df[df["UnitNumber"] == value]
        if len(rows) < sample_size : # si la taille de l'echantillon donnée est superieur au nombre total de ligne pour un moteur, reinitialiser la valeur
            sample_size = len(rows)
        random_sample = rows.sample(n=sample_size, random_state = rand_state)  # Sélectionne 50 lignes aléatoires
        selected_rows.append(random_sample)
    result = pd.concat(selected_rows)
    return result

# **Data normalization**

In [39]:
def normalised_df(train, test):

  from sklearn.preprocessing import MinMaxScaler

  # Instancier l'objet MinMaxScaler pour normaliser les données
  scaler = MinMaxScaler()

  # Normaliser train
  train_scaled = scaler.fit_transform(train)
  train_df = pd.DataFrame(train_scaled)

  # Normaliser test
  test_scaled = scaler.fit_transform(test)
  test_df = pd.DataFrame(test_scaled)


  train_df.columns = train.columns
  test_df.columns = test.columns
    
  return train_df, test_df

# **Data splitting**

In [40]:
def data_split(train, test):

  # data split
  X_train = train.drop('RUL', axis=1)
  Y_train = train['RUL']
  X_test = test.drop('RUL', axis=1)
  Y_test = test['RUL']
    
  return X_train, Y_train, X_test, Y_test


# calcul de S-score

In [41]:
#calcul de S-score
def compute_s_score(rul_true, rul_pred):
    diff = rul_pred - rul_true
    return np.sum(np.where(diff < 0, np.exp(-diff/13)-1, np.exp(diff/10)-1))

# **The XGB Regressor model**

In [42]:
import time

def my_XGB_Regressor(x, y):

    # Time tracking, Operation time (min)
    t = time.process_time()


    mse_test_list = []
    rmse_test_list = []
    mae_test_list = []
    mape_test_list = []
    #s_score_list = []
    
    
    for j in range(1, 11):
        train_selected = selection_aleatoire(x, 50, j)
        test_selected = selection_aleatoire(y, 25, j)

        normalized_train_df, normalized_test_df =  normalised_df(train_selected, test_selected)

        X_train, Y_train, X_test, Y_test = data_split(normalized_train_df, normalized_test_df)

        # Créer le modèle XGBoost Regressor
        model = XGBRegressor()

        # Définir les paramètres à tester dans la recherche par grille
        '''param_grid = {
            'n_estimators': [100, 500, 1000],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.3], #np.logspace(-3,-1,10)
            'subsample': [0.5, 0.7, 0.9],
            'colsample_bytree': [0.5, 0.7, 0.9],
            }'''
        param_grid = {
            'n_estimators': [100],
            'max_depth': [3],
            'learning_rate': [0.01], #np.logspace(-3,-1,10)
            'subsample': [0.5],
            'colsample_bytree': [0.5],
            }

        # Créer l'objet GridSearchCV
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

        # Effectuer la recherche par grille sur les données d'entraînement
        grid_search.fit(X_train, Y_train)

        # Afficher les meilleurs paramètres trouvés
        print("Meilleurs paramètres trouvés :")
        print(grid_search.best_params_)
        
        # Créer l'objet GridSearchCV
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

        # Effectuer la recherche par grille sur les données d'entraînement
        grid_search.fit(X_train, Y_train)

        print("\n***********************TOUR No ",j,"/10:************************")

        # Afficher les meilleurs paramètres trouvés
        print("Meilleurs paramètres trouvés :")
        print(grid_search.best_params_)

        '''#-------------Train---------------------
        # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Train
        y_pred_train = grid_search.predict(X_train)
        # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
        mse_train = mean_squared_error(Y_train, y_pred_train)
        rmse_train = np.sqrt(mse_train)
        print('Train:==========================================================')
        print('MSE : ',mse_train * 100,'%')
        print('RMSE : ',rmse_train * 100,'%')'''

        #-------------Test---------------------
        # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Test
        y_pred_test = grid_search.predict(X_test)
        # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
        mse_test = mean_squared_error(Y_test, y_pred_test)
        rmse_test = np.sqrt(mse_test)
        mae_test = mean_absolute_error(Y_test, y_pred_test)
        mape_test = np.mean(np.abs((Y_test - y_pred_test) / Y_test)) * 100
        s_score = compute_s_score(Y_test, y_pred_test)

        print('\n=============================Test=============================')
        print('MSE : ',mse_test * 100,'%')
        print('RMSE : ',rmse_test * 100,'%')
        print('MAE : ',mae_test * 100,'%')
        print('MAPE : ',mape_test,'%')
        #print("S-score: ", s_score)

        #score
        '''train_rmse_list.append(mse_train)'''
        mse_test_list.append(mse_test)
        rmse_test_list.append(rmse_test)
        mae_test_list.append(mae_test)
        mape_test_list.append(mape_test)
        #s_score_list.append(s_score)
        
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print("time: " , (time.process_time()-t)/60,"min")
        print('')
        
        
        
    print("\n\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Total time: " , (time.process_time()-t)/60,"min")
    
    #print('All S-Score: ',s_score_list)
    #print('\n mean S-Score', np.mean(s_score_list),'\n')
    
    print('All MSE: ',mse_test_list)
    print('mean MSE', np.mean(mse_test_list), " == ",np.mean(mse_test_list)*100,"%\n")
    
    print('All RMSE: ',rmse_test_list)
    print('\n******************************************************************************************')
    print('******************************************************************************************')
    print('***************** mean RMSE', np.mean(rmse_test_list), " ==> ",np.mean(rmse_test_list)*100,"% ***************")
    print('******************************************************************************************')
    print('******************************************************************************************\n')


    print('mean MAE : ', np.mean(mae_test_list) * 100,'%')
    print('mean MAPE : ', np.mean(mape_test_list),'%')

In [43]:
my_XGB_Regressor(train_fd001, test_fd001)

Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

***********************TOUR No  1 /10:************************
Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

=============================Test=============================
MSE :  1.5505445452235704 %
RMSE :  12.452086352188417 %
MAE :  9.575245555145608 %
MAPE :  inf %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  0.16328125 min

Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

***********************TOUR No  2 /10:************************
Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

=============================Test=============================
MSE :  1.8768074499145

In [44]:
my_XGB_Regressor(train_fd002, test_fd002)

Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

***********************TOUR No  1 /10:************************
Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

=============================Test=============================
MSE :  1.7349237205247094 %
RMSE :  13.171650316208328 %
MAE :  10.547893635041701 %
MAPE :  inf %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  0.35130208333333335 min

Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

***********************TOUR No  2 /10:************************
Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

=============================Test=============================
MSE :  1.916

In [45]:
my_XGB_Regressor(train_fd003, test_fd003)

Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

***********************TOUR No  1 /10:************************
Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

=============================Test=============================
MSE :  1.3959414857787695 %
RMSE :  11.814996765885166 %
MAE :  8.706080382284087 %
MAPE :  inf %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  0.16614583333333333 min

Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

***********************TOUR No  2 /10:************************
Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

=============================Test=============================
MSE :  1.7119

In [46]:
my_XGB_Regressor(train_fd004, test_fd004)

Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

***********************TOUR No  1 /10:************************
Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

=============================Test=============================
MSE :  1.6933858035777622 %
RMSE :  13.013015805637687 %
MAE :  9.973848179595745 %
MAPE :  inf %
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
time:  0.2838541666666667 min

Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

***********************TOUR No  2 /10:************************
Meilleurs paramètres trouvés :
{'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}

=============================Test=============================
MSE :  1.76067